In [1]:
!pip install tensorflow-gpu==2.0.0-rc2

In [0]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model 
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.datasets import mnist

In [0]:
class Generator(keras.layers.Layer):
  def __init__(self, input_dim=100, output_dim=784, name="Generator", **kwargs):
    super(Generator, self).__init__(name=name, **kwargs)

    self.input_dim = input_dim
    self.output_dim = output_dim
    
    self.hidden = list()
    self.hidden.append(Dense(units=256,input_dim=input_dim, activation=tf.nn.leaky_relu,  name='generator_input'))
    self.hidden.append(Dense(units=512, activation=tf.nn.leaky_relu))
    self.hidden.append(Dense(units=1024, activation=tf.nn.leaky_relu))
    self.output_layer = Dense(units=output_dim, activation='tanh', name='generator_output')

  def call(self, inputs):
    x = self.hidden[0](inputs)
    for layer in self.hidden[1:]:
      x = layer(x)
    return self.output_layer(x)
  

In [0]:
class Discriminator(keras.layers.Layer):
  def __init__(self, input_dim=784, name="Discriminator", **kwargs):
    super(Discriminator, self).__init__(name=name, **kwargs)

    self.input_dim = input_dim
    
    self.hidden = list()
    self.hidden.append(Dense(units=1024,input_dim=input_dim, activation=tf.nn.leaky_relu,  name='discriminator_input'))
    self.hidden.append(Dropout(0.2))
    self.hidden.append(Dense(units=512, activation=tf.nn.leaky_relu))
    self.hidden.append(Dropout(0.2))
    self.hidden.append(Dense(units=256, activation=tf.nn.leaky_relu))
    self.output_layer = Dense(units=1, activation='sigmoid', name='discriminator_output')

  def call(self, inputs):
    x = self.hidden[0](inputs)
    for layer in self.hidden[1:]:
      x = layer(x)
    return self.output_layer(x)

In [0]:
class GAN(keras.Model):
  def __init__(self, input_dim=100, latent_dim=784, name="GAN", **kwargs):
    super(GAN, self).__init__(name=name, **kwargs)
    self.input_dim = input_dim
    self.latent_dim = latent_dim
    self.G = Generator(latent_dim, input_dim)
    self.D = Discriminator(input_dim)
  
  def call(self, input):
    x = self.G(input)
    x = tf.concat([x, input[1]])
    x = self.D(x)
    return x
  

In [0]:
class GANback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    self.model.D.trainable = !self.model.D.trainable
callback = GANback()

In [0]:
(X, _), (X_test, _) = mnist.load_data()
X = (X.astype(np.float32) - 127.5)/127.5
X = X.reshape(60000, 784)
y = np.zeros(shape=X)

In [0]:
batch_size=128
model = GAN()
model.compile(loss='binary_crossentropy', optimizer='adam')

In [37]:
noise = np.random.normal(0,1, [batch_size, 100])
original_images = X[np.random.randint(low=0, high=X.shape[0], size=batch_size)]
img = [noise, original_images]
y_dis = np.zeros(2*batch_size)
y_dis[batch_size:] = 0.9
model.train_on_batch(img, y_dis)

ValueError: ignored